<a href="https://colab.research.google.com/github/thavha/Regression-predict-group-5/blob/master/latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Data Preprocessing**
Importing Liabraries


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import datetime

**Importing the data set** and checking the data

In [159]:
dataset = pd.read_csv('Train.csv')
dataset_test = pd.read_csv('Test.csv')
dataset.head()
#dataset_test.head()


,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Confirmation - Time,Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Arrival at Pickup - Time,Pickup - Day of Month,Pickup - Weekday (Mo = 1),Pickup - Time,Arrival at Destination - Day of Month,Arrival at Destination - Weekday (Mo = 1),Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,9:35:46 AM,9,5,9:40:10 AM,9,5,10:04:47 AM,9,5,10:27:30 AM,9,5,10:39:55 AM,4,20.4,NaN,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745
1,Order_No_25375,User_Id_2285,Bike,3,Personal,12,5,11:16:16 AM,12,5,11:23:21 AM,12,5,11:40:22 AM,12,5,11:44:09 AM,12,5,12:17:22 PM,16,26.4,NaN,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993
2,Order_No_1899,User_Id_265,Bike,3,Business,30,2,12:39:25 PM,30,2,12:42:44 PM,30,2,12:49:34 PM,30,2,12:53:03 PM,30,2,1:00:38 PM,3,NaN,NaN,-1.308284,36.843419,-1.300921,36.828195,Rider_Id_155,455
3,Order_No_9336,User_Id_1402,Bike,3,Business,15,5,9:25:34 AM,15,5,9:26:05 AM,15,5,9:37:56 AM,15,5,9:43:06 AM,15,5,10:05:27 AM,9,19.2,NaN,-1.281301,36.832396,-1.257147,36.795063,Rider_Id_855,1341
4,Order_No_27883,User_Id_1737,Bike,1,Personal,13,1,9:55:18 AM,13,1,9:56:18 AM,13,1,10:03:53 AM,13,1,10:05:23 AM,13,1,10:25:37 AM,9,15.4,NaN,-1.266597,36.792118,-1.295041,36.809817,Rider_Id_770,1214


selecting only the rlevent  columns that will be used for the model and replacing white space with _ on columns names

In [0]:
new_df = dataset[['Order No','User Id','Platform Type','Personal or Business','Placement - Day of Month','Placement - Weekday (Mo = 1)','Placement - Time','Confirmation - Time','Arrival at Pickup - Time','Arrival at Destination - Time','Distance (KM)','Temperature','Rider Id','Time from Pickup to Arrival']].copy()
new_df.columns = [col.replace(" ","_") for col in new_df.columns] 
dataset_test  = dataset[['User Id','Platform Type','Personal or Business','Placement - Day of Month','Placement - Weekday (Mo = 1)','Placement - Time','Confirmation - Time','Arrival at Pickup - Time','Arrival at Destination - Time','Distance (KM)','Temperature','Rider Id','Time from Pickup to Arrival']].copy()
dataset_test.columns = [col.replace(" ","_") for col in dataset_test.columns] 


encoding personal or business column


In [0]:
new_df = pd.get_dummies(new_df, columns=['Personal_or_Business'], prefix = ['personal'])
new_df = new_df.iloc[:, :-1]
new_df.columns = [*new_df.columns[:-1], 'Business']


dataset_test = pd.get_dummies(dataset_test, columns=['Personal_or_Business'], prefix = ['personal'])
dataset_test = dataset_test.iloc[:, :-1]
dataset_test.columns = [*dataset_test.columns[:-1], 'Business']

dealing with the missing valuse replace with the mean/median temperature 

In [0]:
new_df['Temperature'] = new_df.Temperature.fillna(new_df.Temperature.median())


dataset_test['Temperature'] = dataset_test.Temperature.fillna(dataset_test.Temperature.median())


dealing with time columns

In [0]:
new_df['Placement_-_Time 24'] = (pd.to_datetime(new_df['Placement_-_Time'].str.strip(), format='%I:%M:%S %p')
                  .dt.strftime('%H:%M'))
new_df['Confirmation_-_Time 24'] = (pd.to_datetime(new_df['Confirmation_-_Time'].str.strip(), format='%I:%M:%S %p')
                  .dt.strftime('%H:%M'))
new_df['Arrival_at_Pickup_-_Time 24'] = (pd.to_datetime(new_df['Arrival_at_Pickup_-_Time'].str.strip(), format='%I:%M:%S %p')
                  .dt.strftime('%H:%M'))
new_df['Arrival_at_Destination_-_Time 24'] = (pd.to_datetime(new_df['Arrival_at_Destination_-_Time'].str.strip(), format='%I:%M:%S %p')
                  .dt.strftime('%H:%M'))



dataset_test['Placement_-_Time 24'] = (pd.to_datetime(new_df['Placement_-_Time'].str.strip(), format='%I:%M:%S %p')
                  .dt.strftime('%H:%M'))
dataset_test['Confirmation_-_Time 24'] = (pd.to_datetime(new_df['Confirmation_-_Time'].str.strip(), format='%I:%M:%S %p')
                  .dt.strftime('%H:%M'))
dataset_test['Arrival_at_Pickup_-_Time 24'] = (pd.to_datetime(new_df['Arrival_at_Pickup_-_Time'].str.strip(), format='%I:%M:%S %p')
                  .dt.strftime('%H:%M'))
dataset_test['Arrival_at_Destination_-_Time 24'] = (pd.to_datetime(new_df['Arrival_at_Destination_-_Time'].str.strip(), format='%I:%M:%S %p')
                  .dt.strftime('%H:%M'))

In [61]:
new_df.drop(['Placement_-_Time', 'Confirmation_-_Time','Arrival_at_Pickup_-_Time','Arrival_at_Destination_-_Time','User_Id','Rider_Id'], axis=1)

dataset_test.drop(['Placement_-_Time', 'Confirmation_-_Time','Arrival_at_Pickup_-_Time','Arrival_at_Destination_-_Time','User_Id','Rider_Id'], axis=1)

,Platform_Type,Placement_-_Day_of_Month,Placement_-_Weekday_(Mo_=_1),Distance_(KM),Temperature,Time_from_Pickup_to_Arrival,Business,Placement_-_Time 24,Confirmation_-_Time 24,Arrival_at_Pickup_-_Time 24,Arrival_at_Destination_-_Time 24
0,3,9,5,4,20.4,745,1,09:35,09:40,10:04,10:39
1,3,12,5,16,26.4,1993,0,11:16,11:23,11:40,12:17
2,3,30,2,3,23.5,455,1,12:39,12:42,12:49,13:00
3,3,15,5,9,19.2,1341,1,09:25,09:26,09:37,10:05
4,1,13,1,9,15.4,1214,0,09:55,09:56,10:03,10:25
...,...,...,...,...,...,...,...,...,...,...,...
21196,3,20,3,3,28.6,9,0,15:54,15:55,15:58,16:20
21197,3,13,6,7,26.0,770,1,10:13,10:13,10:20,10:46
21198,3,7,4,20,29.2,2953,1,17:06,17:07,17:30,18:40
21199,1,4,3,13,15.0,1380,0,09:31,09:31,09:38,10:08


spliting data into X and Y , first converting time into hours lapse since the day started

In [0]:
new_df = new_df[['Platform_Type', 'Arrival_at_Pickup_-_Time 24','Confirmation_-_Time 24','Placement_-_Time 24','Business','Temperature','Distance_(KM)','Placement_-_Weekday_(Mo_=_1)','Placement_-_Day_of_Month','Platform_Type', 'Arrival_at_Destination_-_Time 24', 'Time_from_Pickup_to_Arrival']]
new_df['Placement_-_Time 24'] = new_df['Placement_-_Time 24'].str.replace(':','.')
new_df['Confirmation_-_Time 24'] = new_df['Confirmation_-_Time 24'].str.replace(':','.')
new_df['Arrival_at_Pickup_-_Time 24'] = new_df['Arrival_at_Pickup_-_Time 24'].str.replace(':','.')
new_df['Arrival_at_Destination_-_Time 24'] = new_df['Arrival_at_Destination_-_Time 24'].str.replace(':','.')
new_df.head()


dataset_test = dataset_test[['Platform_Type', 'Arrival_at_Pickup_-_Time 24','Confirmation_-_Time 24','Placement_-_Time 24','Business','Temperature','Distance_(KM)','Placement_-_Weekday_(Mo_=_1)','Placement_-_Day_of_Month','Platform_Type', 'Arrival_at_Destination_-_Time 24', 'Time_from_Pickup_to_Arrival']]
dataset_test['Placement_-_Time 24'] = dataset_test['Placement_-_Time 24'].str.replace(':','.')
dataset_test['Confirmation_-_Time 24'] = dataset_test['Confirmation_-_Time 24'].str.replace(':','.')
dataset_test['Arrival_at_Pickup_-_Time 24'] = dataset_test['Arrival_at_Pickup_-_Time 24'].str.replace(':','.')
dataset_test['Arrival_at_Destination_-_Time 24'] = dataset_test['Arrival_at_Destination_-_Time 24'].str.replace(':','.')



In [0]:
x =pd.DataFrame(new_df.iloc[:,:-1].values)
y = new_df.iloc[:,-1].values


x_test =pd.DataFrame(new_df.iloc[:,:-1].values)
y_test= new_df.iloc[:,-1].values

Rescaling the x values using standardisation

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(x)
x_standardise = pd.DataFrame(X_scaled,columns=x.columns)
#x_standardise.head()



In [0]:

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_scaledtest = scaler.fit_transform(x_test)
x_standardisetest = pd.DataFrame(x_scaledtest,columns=x.columns)
#x_standardisetest.head()

training a model using lasso

In [0]:
#from sklearn.linear_model import Lasso
#lasso = Lasso(alpha=0.01)
#lasso.fit(X_standardise, y)

In [0]:
#from sklearn import metrics
#train_lasso = lasso.predict(X_standardise)
#test_lasso = lasso.predict(X_test)

In [127]:
l=np.array(y_test)
type(l)

numpy.ndarray

training using simple linear 

In [68]:
from sklearn.linear_model import LinearRegression
regressor =LinearRegression()
regressor.fit(x, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [125]:
y_pred= regressor.predict(x)
np.set_printoptions(precision=2)
f=pd.DataFrame(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_pred),1)), 1))
#f.to_csv('submission1.csv',index=False,columns=['Order_No','Time from Pickup to Arrival']))
b=f[[1]]
b.head()

,1
0,745.0
1,1993.0
2,455.0
3,1341.0
4,1214.0


In [0]:
o= np.array(dataset['Order No'])
final = np.vstack([o,l]).T
r=pd.DataFrame(final)
#r.rename(columns={'0': 'Order_No', '1':'Time from Pickup to Arrival'})

r.to_csv('submision.csv', index=False)

In [163]:
dataset_ = pd.read_csv('submision.csv')
dataset_.head()


,0,1
0,Order_No_4211,745
1,Order_No_25375,1993
2,Order_No_1899,455
3,Order_No_9336,1341
4,Order_No_27883,1214


In [156]:
o

array(['Order_No_4211', 'Order_No_25375', 'Order_No_1899', ...,
       'Order_No_2831', 'Order_No_6174', 'Order_No_9836'], dtype=object)